### **Fine Tunning de Modelo de IA**

O notebook faz a leitura de um arquivo JSON com perguntas e respostas, converte cada entrada para o formato esperado por modelos de linguagem (JSONL com mensagens de usuário e assistente), e grava tudo em um novo arquivo `.jsonl` pronto para ser usado em fine-tuning.

**1. Importação do módulo JSON**

- Importa o módulo `json`, que permite ler e escrever arquivos no formato JSON em Python.

In [29]:
import json


**2. Leitura do arquivo de respostas em JSON**
- Abre o arquivo `chatbot_respostas.json` para leitura, usando o encoding `utf-8` (necessário para suportar acentos e caracteres especiais).
- Carrega o conteúdo do arquivo para a variável `json_respostas`, que se torna uma lista de dicionários Python.
- Cada dicionário representa uma pergunta, resposta e categoria.

In [30]:
json_respostas = []
with open('C:/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/chatbot_respostas.json', encoding="utf8") as f:
    json_respostas = json.load(f)

**3. Visualização dos dados carregados**
- Exibe o conteúdo da variável `json_respostas` na tela do notebook.
- Útil para conferir se os dados foram carregados corretamente e visualizar a estrutura dos dados.


In [31]:
json_respostas

[{'pergunta': 'Qual é a fórmula da área de um círculo?',
  'resposta': 'A área de um círculo é π vezes o raio ao quadrado (A = πr²).',
  'categoria': 'Matemática'},
 {'pergunta': 'O que é um substantivo?',
  'resposta': 'Substantivo é a classe gramatical de palavras que nomeiam seres, objetos, fenômenos, lugares, qualidades e ações.',
  'categoria': 'Língua Portuguesa'},
 {'pergunta': 'O que é inércia?',
  'resposta': 'Inércia é a tendência de um corpo manter seu estado de repouso ou movimento uniforme, a menos que forças atuem sobre ele.',
  'categoria': 'Física'},
 {'pergunta': 'O que é uma equação do segundo grau?',
  'resposta': 'É uma equação polinomial onde o maior expoente da incógnita é 2, geralmente na forma ax² + bx + c = 0.',
  'categoria': 'Matemática'},
 {'pergunta': 'O que é fotossíntese?',
  'resposta': 'Fotossíntese é o processo pelo qual plantas e outros organismos convertem luz solar em energia química para alimentar suas atividades.',
  'categoria': 'Biologia'},
 {'p

**4. Escrita dos dados em formato JSONL (primeira tentativa)**

- Tenta criar um novo arquivo `chatbot_respostas.jsonl` no formato JSONL (um JSON por linha).
- Para cada entrada em `json_respostas`, monta um novo dicionário no formato esperado para fine-tuning de chatbots (com campos `messages`, `role` e `content`).
- Adiciona a fonte "AsimoBot" em cada resposta.


In [32]:
with open('C:/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/chatbot_respostas.jsonl','w', encoding='utf-8') as f:
    for entrada in json_respostas: 
        resposta = {
        'resposta': entrada['resposta'],
        'categoria': entrada['categoria'],
        'fonte': 'AsimoBot'
        }

    entrada_jsonl = {
            'messages': [
                {'role': 'user', 'content': entrada['pergunta']},
                {'role': 'assistant', 'content': json.dumps(resposta, ensure_ascii=False, indent=2)}
            ]
    }

    json.dump(entrada_jsonl, f, ensure_ascii=False)
    f.write('\n')
  

**5. Escrita dos dados em formato JSONL (versão corrigida)**

- Corrige o problema do arquivo JSON, garantindo que cada entrada seja gravada corretamente no arquivo, uma por linha.
- Para cada pergunta/resposta, grava um objeto JSONL no formato adequado para datasets de fine-tuning de chatbots.
- Usa `encoding='utf-8'` para evitar problemas com acentos e caracteres especiais.
- Usa `ensure_ascii=False` para manter os caracteres especiais no arquivo.

In [33]:
with open('C:/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/chatbot_respostas.jsonl','w', encoding='utf-8' ) as f:
    for entrada in json_respostas:
        resposta = {
            'resposta': entrada['resposta'],
            'categoria': entrada['categoria'],
            'fonte': 'AsimoBot'
        }
        entrada_jsonl = {
            'messages': [
                {'role': 'user', 'content': entrada['pergunta']},
                {'role': 'assistant', 'content': json.dumps(resposta, ensure_ascii=False, indent=2)}
            ]
        }
        json.dump(entrada_jsonl, f, ensure_ascii=False)
        f.write('\n')

**6. Criar Client e importar chaves da API**

Carrega a chave da API do arquivo .env e Cria o cliente da OpenAI (autentica automaticamente com a variável OPENAI_API_KEY) 

In [34]:
import openai
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

client = openai.Client()

**7. Enviar arquivo jsonl para fine tunning na API da OpenAI**

Esse trecho de código em Python utiliza a biblioteca openai (representada aqui como client) para criar um arquivo de treinamento e iniciar um job de fine-tuning (ajuste fino) em um modelo da OpenAI. Vamos detalhar passo a passo:

In [ ]:
# Envia o arquivo chatbot_respostas.jsonl para uso no treinamento.
ile = client.files.create(
    file=open('C:/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/chatbot_respostas.jsonl', 'rb'),
    purpose = 'fine-tune'
)

# Inicia o treinamento do modelo com esse arquivo.
client.fine_tuning.jobs.create(
    training_file = file.id,
    model='gpt-3.5-turbo'
)



FineTuningJob(id='ftjob-nVueL8JLqBnUrHpCenQG0cdH', created_at=1752096525, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-N6fe9JXaAIovE9oO3HPcxbAi', result_files=[], status='validating_files', trained_tokens=None, training_file='file-6jEiqZtgmpeu1ob1Py9UpH', validation_file=None, user_provided_suffix=None, seed=823466306, estimated_finish=None, integrations=[], metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

Esta parte acessa a parte da API que gerencia os treinamentos personalizados.

.list(): retorna uma lista de todos os jobs de fine-tuning criados, com informações como:

ID do job

- Status (em andamento, concluído, com erro etc.)

- Modelo usado

- Arquivo de treinamento

- Datas de início/fim'''

In [36]:

client.fine_tuning.jobs.list()



SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-nVueL8JLqBnUrHpCenQG0cdH', created_at=1752096525, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=1752097311, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-N6fe9JXaAIovE9oO3HPcxbAi', result_files=[], status='running', trained_tokens=None, training_file='file-6jEiqZtgmpeu1ob1Py9UpH', validation_file=None, user_provided_suffix=None, seed=823466306, estimated_finish=None, integrations=[], metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}}), FineTuningJob(id='ftjob-f48hGcWU2vu49XceKMmBBkee', created_at=1752096511, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=1752097284, hyperparameters=Hy

# **Utilizando o Modelo Refinado**

In [ ]:
mensagens = [{'role':'user', 'content':'O que é uma função quadrática?'}]

resposta = client.chat.completions.create(
    messages= mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-3-turbo` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}